In [90]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.preprocessing import OneHotEncoder
from scipy import stats


%matplotlib inline

# Data Management

In [91]:
# group data by activity
def data_by_activity(X, y, activities):
    # group windows by activity
    grouped=list()
    #return {a:X[y[:,0]==a] for a in activities}
    for i in activities:
        ac = X[Y[:,0]==i]
        grouped.append(ac)
        #print(i,ac.shape)

def load_all_data(directory):
    # Load
    filename="mHealth_subject1.csv"
    df = pd.read_csv(directory+filename)
    df.insert(0, 'id', 1)


    for i in range(9):
        number=str(i+2)
        filename="mHealth_subject"+number+".csv"
    #   print(directory+filename)
        df_subject = pd.read_csv(directory+filename)
        df_subject.insert(0, 'id', i+2)
        df = df.append(df_subject)



    # Cleaning
    df =df.query('label != 0')
    raw = df
    
    # Separate data
    X = df.iloc[:, :24]
    Y = df.iloc[:,24]

    return raw,X,Y


def class_breakdown(data):
    # convert the numpy array into a dataframe
    df = pd.DataFrame(data)
    # group data by the class value and calculate the number of rows
    counts = df.groupby(0).size()
    # retrieve raw rows
    counts = counts.values
    # summarize
    for i in range(len(counts)):
        percent = counts[i] / len(df) * 100
        print('Class=%d, total=%d, percentage=%.3f' % (i + 1, counts[i], percent))


# Method to convert data to series
def to_series(data, off, activity_list, subject_id):
    subject_data = data.query('id==' + str(subject_id))
    series = [[]]
    for activity in activity_list:
        ser = np.asmatrix(subject_data.query("label=="+str(activity)).iloc[:, off]).T
        series=np.append(series,ser)
    return series


def min_max_normalization(X):
    row,columns=X.shape
    for i in range(columns):
        v = X[:, i]  
        X[:, i] = (v - v.min()) / ((v.max() - v.min()) if (v.max() - v.min())!=0 else 1)
    return X

# Scale dataset in a range
def range_normalization(X,a,b):
    row,columns=X.shape
    for i in range(columns):
        v = X[:, i]  
        X[:, i] = (b-a)*((v - v.min()) / ((v.max() - v.min()) if (v.max() - v.min())!=0 else 1))+a
    return X


# Create a dataset as a time sequence
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)


# Encode Target as a vector
def encode_target(Y):
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    enc = enc.fit(Y)
    encode_target = enc.transform(Y)
    return encode_target

# Utils

In [92]:
import numpy as np
import random
import sys


def sigmoid(x):
    return 1. / (1 + np.exp(-x))


def sigmoid_derivative(values):
    return values * (1 - values)

def tanh_normal(values):
    return np.tanh(values)

def tanh_derivative(values):
    return 1. - values ** 2


def softmax(X):
    exp_X = np.exp(X)
    exp_X_sum = np.sum(exp_X,axis=1).reshape(-1,1)
    exp_X = exp_X/exp_X_sum
    return exp_X


# createst uniform random array w/ values in [a,b) and shape args
def rand_arr(a, b, *args):
    np.random.seed(0)
    return np.random.rand(*args) * (b - a) + a


# Creates a list of tuples wit mini batches for X and Y
def get_mini_batches(X, y, batch_size):
    random_idxs = np.random.choice(len(y), len(y), replace=False)
    X_shuffled = X[random_idxs, :, :]
    y_shuffled = y[random_idxs,:]
    mini_batches = [(X_shuffled[i:i + batch_size, :,:], y_shuffled[i:i + batch_size,:]) for i in range(0, len(y), batch_size)]
    return mini_batches

def get_index_batch(batch,range_low, range_high):
    seedValue = (random.randrange(sys.maxsize))//2**32
    np.random.seed(seedValue)
    return np.random.randint(low=range_low, high=range_high, size=(batch,))



# LSTM

In [93]:
# Class params will contain all parameters of the network 
#import numpy as np


class lstm_param:

    def __init__(self, units, x_dim, classes):
        self.units = units
        self.x_dim = x_dim
        concat_len = x_dim + units
        # weight matrices
        self.wg = rand_arr(-0.1, 0.1, units, concat_len)
        self.wi = rand_arr(-0.1, 0.1, units, concat_len)
        self.wf = rand_arr(-0.1, 0.1, units, concat_len)
        self.wo = rand_arr(-0.1, 0.1, units, concat_len)
        self.wk = rand_arr(-0.1, 0.1, units, classes)
        
        # bias terms
        self.bg = rand_arr(-0.1, 0.1, units,  1)
        self.bi = rand_arr(-0.1, 0.1, units,  1)
        self.bf = rand_arr(-0.1, 0.1, units,  1)
        self.bo = rand_arr(-0.1, 0.1, units,  1)
        self.bk = rand_arr(-0.1, 0.1, classes,1)

        # diffs (derivative of loss function w.r.t. all parameters)
        self.wg_diff = np.zeros((units, concat_len))
        self.wi_diff = np.zeros((units, concat_len))
        self.wf_diff = np.zeros((units, concat_len))
        self.wo_diff = np.zeros((units, concat_len))
        self.wk_diff = np.zeros((units, classes))
        
        self.bg_diff = np.zeros((units,  1))
        self.bi_diff = np.zeros((units,  1))
        self.bf_diff = np.zeros((units,  1))
        self.bo_diff = np.zeros((units,  1))
        self.bk_diff = np.zeros((classes,1))

    def apply_diff(self, lr=1):
        self.wg -= lr * self.wg_diff
        self.wi -= lr * self.wi_diff
        self.wf -= lr * self.wf_diff
        self.wo -= lr * self.wo_diff
        self.wk -= lr * self.wk_diff
        
        self.bg -= lr * self.bg_diff
        self.bi -= lr * self.bi_diff
        self.bf -= lr * self.bf_diff
        self.bo -= lr * self.bo_diff
        self.bk -= lr * self.bk_diff
        
        # reset diffs to zero
        self.wg_diff = np.zeros_like(self.wg)
        self.wi_diff = np.zeros_like(self.wi)
        self.wf_diff = np.zeros_like(self.wf)
        self.wo_diff = np.zeros_like(self.wo)
        self.wk_diff = np.zeros_like(self.wk)
        
        self.bg_diff = np.zeros_like(self.bg)
        self.bi_diff = np.zeros_like(self.bi)
        self.bf_diff = np.zeros_like(self.bf)
        self.bo_diff = np.zeros_like(self.bo)
        self.bk_diff = np.zeros_like(self.bk)

In [94]:
class lstm_state:
    def __init__(self, units, x_dim,series):
        self.cell_temp_values = np.zeros(units)
        self.input_values     = np.zeros(units)
        self.forget_values    = np.zeros(units)
        self.output_values    = np.zeros(units)
        self.cell_values      = np.zeros(units)
        self.h                = np.zeros((units,series))
        
        self.diff_h_values    = np.zeros_like(self.h)
        self.diff_cell_values = np.zeros_like(self.cell_values)


In [105]:
# from lstm_param import lstm_param

class lstm_model:

    def __init__(self,features, target, classes, units, batch):
        self.X = features
        self.Y = target
        self.classes = classes
        self.units = units
        self.batch_size = batch

        z,x,y=features.shape

        self.x_dim = x
        self.h_dim = x+units
        self.params = lstm_param(units,x,classes)
               

    def train(self, epochs):
        samples, features, series=self.X.shape
        
        for epoch in range(epochs):
            
            # create batches
            batch_set = get_mini_batches(self.X,self.Y,self.batch_size)
            cell_prev = None
            h_prev    = None
            
            # LSTM recurrent
            state = lstm_state(self.units, self.x_dim, series)
            
            
            # Iterations over batchs
            for current_batch in batch_set:
                x_current = current_batch[0]
                y_current = current_batch[1]
                
            
                # Cell state
                if cell_prev is None: cell_prev = np.zeros_like(state.cell_values)
                if h_prev is None: h_prev = np.zeros_like(state.h)  
                loss = 0
                avg_loss = 0
                predicted_values = np.zeros((samples,2,1))
                
                
                # Iterations by value
                for step in range(samples):
                    time_serie = x_current[step]
                    target = np.matrix(y_current[step])
                
                    # Forward
                    predicted, step_loss, state = self.forward_step(time_serie, target, h_prev, cell_prev, state)
                    predicted_values[step] = predicted
                    loss += step_loss
                    
                    h_prev = state.h
                    cell_prev =  state.cell_values
                    break
                    
                # Back
                avg_loss = loss/self.batch_size 
                
                # TODO: 
                    
                    
                
                
                break
            break
        
                        
            
        
                
    # x = time series matrix
    # h_prev = hidden
    def forward_step(self, x, target, h_prev, cell_prev, state):
        prob=1
        
        #LSTM
        xc = np.vstack((x, h_prev))
        state.cell_temp_values = np.tanh(np.dot(self.params.wg, xc) + self.params.bg)
        state.input_values = sigmoid(np.dot(self.params.wi, xc) + self.params.bi)
        state.forget_values = sigmoid(np.dot(self.params.wf, xc) + self.params.bf)
        state.output_values = sigmoid(np.dot(self.params.wo, xc) + self.params.bo)
        state.cell_values = state.cell_temp_values * state.input_values + h_prev * state.forget_values
        state.h = state.cell_values * state.output_values
        
        # Classification
        predicted = softmax(np.dot(self.params.wk.T,state.h)+self.params.bk)
        prob = np.multiply(prob,np.sum(np.multiply(target.T,predicted),axis=1).reshape(-1,1))
        result = np.sum(prob,axis=1,dtype="float32")
        
        # Cross entropy loss
        loss = (-1)*(np.sum((np.dot(target,np.log(predicted)) + np.dot(1-target,np.log(1-predicted))),axis=1).reshape(-1,1))

        return result, loss, state

# Implementation

In [17]:
#Hyper parameters

# Variable to save the number of steps in the sequence
TIME_STEPS = 128
# Step to recolect samples every STEP times a new sample of TIME_STEP will be recolected
STEP = 40

# Units in NN
UNITS = 100

# No. of classes for classification
CLASSES = 2

# Batch size
BATCH_SIZE = 64

In [11]:
# Load Data
directory="Data/MHEALTHDATASET/"

raw,X,y=load_all_data(directory)
raw.head()

# to beginning we are only going to use accelerometer
valid_activities_set=raw.query("label==1 or label==4") #raw.copy()
valid_activities_set.columns

# Get only 2 activities and accelerometer data
# data=valid_activities_set[['id','acc_chest_x'    , 'acc_chest_y'    , 'acc_chest_z','label']].copy()
data=valid_activities_set[['id','acc_chest_x'    , 'acc_chest_y'    , 'acc_chest_z',
                               'acc_left_ank_x' , 'acc_left_ank_y' , 'acc_left_ank_z',
                               'acc_right_arm_x', 'acc_right_arm_y', 'acc_right_arm_z','label']].copy()


# Separate in train and test
df_train = data[data['id'] <= 7]
df_test = data[data['id'] > 7]


# Scale data [-1,1] with min_max
scale_columns = ['acc_chest_x'    , 'acc_chest_y'    , 'acc_chest_z',
                               'acc_left_ank_x' , 'acc_left_ank_y' , 'acc_left_ank_z',
                               'acc_right_arm_x', 'acc_right_arm_y', 'acc_right_arm_z']

df_train.loc[:, scale_columns] = range_normalization(df_train[scale_columns].to_numpy(),-1,1)
df_test.loc[:, scale_columns] = range_normalization(df_test[scale_columns].to_numpy(),-1,1)


# Create dataset as time series
X_train, y_train = create_dataset(
    df_train[['acc_chest_x'    , 'acc_chest_y'    , 'acc_chest_z',
                               'acc_left_ank_x' , 'acc_left_ank_y' , 'acc_left_ank_z',
                               'acc_right_arm_x', 'acc_right_arm_y', 'acc_right_arm_z']], 
    df_train.label, 
    TIME_STEPS, 
    STEP
)

X_test, y_test = create_dataset(
    df_test[['acc_chest_x'    , 'acc_chest_y'    , 'acc_chest_z',
                               'acc_left_ank_x' , 'acc_left_ank_y' , 'acc_left_ank_z',
                               'acc_right_arm_x', 'acc_right_arm_y', 'acc_right_arm_z']], 
    df_test.label, 
    TIME_STEPS, 
    STEP
)


print("Before encode:")
print("Train")
print(X_train.shape, y_train.shape)
print("Test")
print(X_test.shape, y_test.shape)

# Encode Target
y_train = encode_target(y_train)
y_test = encode_target(y_test)

print("After encode:")
print("Train")
print(X_train.shape, y_train.shape)
print("Test")
print(X_test.shape, y_test.shape)

/home/liliana/.local/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Before encode:
Train
(1072, 128, 9) (1072, 1)
Test
(458, 128, 9) (458, 1)
After encode:
Train
(1072, 128, 9) (1072, 2)
Test
(458, 128, 9) (458, 2)


In [106]:
har_lst=lstm_model(X_train, y_train, CLASSES, UNITS, BATCH_SIZE)



har_lst.train(100)

[[20.83791584]]


In [47]:
print(har_lst.params.wo.shape)

(100, 228)
